# Find, Load, and Visualise Earth Observation Imagery

This notebook demonstrates how to find, load, and visualise Earth observation imagery using cloud native approaches, which work well on your desktop or in cloud environments!

The notebook demonstrates how to find images from the Digital Earth Australia STAC catalog, load them using `odc-stac`, and visualise them using extensions to xarray provided by `odc-geo`.

## Set up

The first step is to set up the required Python libraries and local imports.

* `odc.stac` and `pystac_client` are used to access Digital Earth Australia's STAC catalog
* `numpy` is used to manipulate data
* `odc.geo.xr` enables additional functionality for xarrays

In [ ]:
from odc.stac import configure_rio, load
from pystac_client import Client as PystacClient
import numpy as np
import odc.geo.xr

The second step is to start a Dask client.

Dask supports local parallel processing and can help speed up computation times.

In [ ]:
from dask.distributed import Client as DaskClient

dask_client = DaskClient()
dask_client

## Part 1: Find

### 1.1 Connecting to the catalog

In [ ]:
# The catalog URL for DEA's STAC catalog
catalog = "https://explorer.dea.ga.gov.au/stac"

# pystac_client is used to connect to the catalog
stac_client = PystacClient.open(catalog)

# configure settings for reading from DEA's STAC
configure_rio(
    cloud_defaults=True,
    aws={"aws_unsigned": True},
)

### 1.2 Selecting an area to query

TODO: Replace with map selector

In [ ]:
from odc.geo.geom import Geometry

geom = Geometry({'type': 'Polygon', 'coordinates': (((128.29834, -22.73819), (128.29834, -21.981255), (129.276123, -21.981255), (129.276123, -22.73819), (128.29834, -22.73819)),)}, crs="EPSG:4326")


### 1.3 Set year and month to query

In [ ]:
# Set a start and end date
date_query = "2021-12"

### 1.4 Choose collections and filters

In [ ]:
# Set product ID as the STAC "collection"
collections_query = ["ga_s2am_ard_3", "ga_s2bm_ard_3"]

# Set up a filter query
filter_query = "s2cloudless:cloud < 20"

### 1.5 Running the query to indentify matching STAC items

In [ ]:
# Query with filtering
query = stac_client.search(
    intersects=geom,
    collections=collections_query,
    datetime=date_query,
    filter=filter_query,
)

items = list(query.items())

print(f"Found {len(items)} items")

## Load

### 2.1 Using odc-stac to load identified items

Area used by default takes around 2.5 mins on local machine.

In [ ]:
# Load our filtered data
ds_filtered = odc.stac.load(
    query.items(),
    bands=["nbart_red", "nbart_green", "nbart_blue"],
    crs="utm",
    chunks={},
    resolution=30,
    groupby="solar_day",
    geopolygon=geom,
).load()


### 2.2 Review loaded imagery

Identify which image you want to export and note the date.

In [ ]:
ds_filtered.to_array().plot.imshow(col="time", col_wrap=3)

## Part 3: Visualise

### 3.1 Select best image and improve contrast

Use percentile stretching to improve the contrast of the image.

In [ ]:
percentile_stretch = (1, 99)

image_date = "2021-12-20"

best = ds_filtered.sel(time=image_date).squeeze()
rgb_array = best.to_array().values

vmin, vmax = np.nanpercentile(rgb_array, percentile_stretch)

visualisation = best.odc.to_rgba(vmin=vmin, vmax=vmax)

### 3.2 Display on an interactive map

In [ ]:

visualisation.odc.explore()

### 3.3 Export to a cloud-optimised GeoTIFF

In [ ]:
visualisation.odc.write_cog("sentinel2_example.tif", overwrite=True)